# RGI scratchpad

In [ ]:
from rgi.games.connect4 import Connect4
from rgi.players.random_player import RandomPlayer
from rgi.players.human_player import HumanPlayer
from rgi.core.game_runner import run_match

game = Connect4()
players = [HumanPlayer(), RandomPlayer()]

average_rewards = run_match(game, players, num_games=3)
print(f"Average rewards: {average_rewards}")